In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from itertools import product
from sklearn.decomposition import PCA, KernelPCA
from sklearn.manifold import TSNE

from tensorflow.keras import regularizers
from tensorflow.keras.datasets import mnist, cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Activation, BatchNormalization, ReLU, Add, Conv2D, MaxPool2D, AveragePooling2D, ZeroPadding2D
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping, Callback
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow.keras.layers.experimental.preprocessing import RandomCrop, RandomFlip, RandomRotation
from tensorflow.keras.regularizers import l2


In [10]:
# 데이터 불러오기, 정리하기

from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

y_train = y_train.squeeze()
y_test = y_test.squeeze()

print(X_train.shape, y_train.shape,X_test.shape, y_train.shape, y_test.shape)

(50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (50000,) (10000,)


In [44]:
# 러닝 레이트 콜백 함수 정의

def scheduler (epoch, lr):
    guide_epoch = [20,40]
    decay_factor = 0.1
    if epoch in guide_epochs:
        lr = lr * decay_factor
        
def scheduler (guide_epochs, decay_factor):
    def lr_scheduler (epoch,lr):
        if epoch in guide_epochs:
            lr = lr * decay_factor
        return lr
    return lr_scheduler

#lr_scheduler = scheduler([40,70,90], 0.1)

In [4]:
# 학습 중 그래프 표시하기
# plot 활용하는 콜백 클라스 정의

from IPython.display import clear_output

class PlottingCallback(tf.keras.callbacks.Callback):

    def __init__(self):
        self.logs = {'loss':[], 'val_loss':[], 'acc':[], 'val_acc':[]}
    
    def on_epoch_end(self, epoch, logs=None):
        self.logs['loss'].append(logs['loss'])
        self.logs['val_loss'].append(logs['val_loss'])
        self.logs['acc'].append(logs['acc'])
        self.logs['val_acc'].append(logs['val_acc'])

        clear_output()
        plt.figure(figsize=(8,4), dpi=150)
        
        plt.subplot(1,2,1)
        plt.plot(self.logs['loss'], label='train loss')
        plt.plot(self.logs['val_loss'], label='val loss')
        plt.legend()
        
        plt.subplot(1,2,2)
        plt.plot(self.logs['acc'], label='train acc')
        plt.plot(self.logs['val_acc'], label='val acc')        
        plt.legend()

        plt.show()
        print('Epoch: {}, Loss: {:7.5f}, Acc: {:5.2f}%'.format(epoch, logs['loss'], 100*logs['acc']))

In [45]:
# hyper parameter 선언

optimizer = 'sgd'
lr = 0.1
momentum = 0.9
lr_scheduler = scheduler(guide_epochs=[40,70,90], decay_factor=0.1)
reg = 5e-4

In [46]:
# Resnet18

# 32*32*3
input_ = Input(shape=(32,32,3))

# 32*32*64
x = Conv2D(64, kernel_size=3, padding='same', strides=(1,1), use_bias=False, # 레즈넷은 bias 안 씀
           kernel_regularizer=l2(reg))(input_)
x = BatchNormalization()(x)
x = ReLU()(x)

# 1st block
# 32*32*64
filters = 64
ksize = 3
skip = x
x = Conv2D(filters, kernel_size=ksize, padding='same', strides=(1,1), use_bias=False, kernel_regularizer=l2(reg))(x)
x = BatchNormalization()(x)
x = ReLU()(x)
# 32*32*64
x = Conv2D(filters, kernel_size=ksize, padding='same', strides=(1,1), use_bias=False, kernel_regularizer=l2(reg))(x)
x = BatchNormalization()(x)
x = Add()([x, skip])                                                          # 저장해 놓은 skip을 넣어줌
x = ReLU()(x)

# 32*32*64
skip = x
x = Conv2D(filters, kernel_size=ksize, padding='same', strides=(1,1), use_bias=False, kernel_regularizer=l2(reg))(x)
x = BatchNormalization()(x)
x = ReLU()(x)
# 32*32*64
x = Conv2D(filters, kernel_size=ksize, padding='same', strides=(1,1), use_bias=False, kernel_regularizer=l2(reg))(x)
x = BatchNormalization()(x)
x = Add()([x, skip])
x = ReLU()(x)

# 2nd block
# 16*16*128
filters=128
skip = Conv2D(filters, kernel_size=1, padding='same', strides=(2,2), use_bias=False, kernel_regularizer=l2(reg))(x)   # 32*32 -> 16*16 으로 크기를 조정해야 함
x = Conv2D(filters, kernel_size=ksize, padding='same', strides=(2,2), use_bias=False, kernel_regularizer=l2(reg))(x)
x = BatchNormalization()(x)
x = ReLU()(x)
# 16*16*128
x = Conv2D(filters, kernel_size=ksize, padding='same', strides=(1,1), use_bias=False, kernel_regularizer=l2(reg))(x)
x = BatchNormalization()(x)
x = Add()([x, skip])
x = ReLU()(x)

# 16*16*128
skip = x
x = Conv2D(filters, kernel_size=ksize, padding='same', strides=(1,1), use_bias=False, kernel_regularizer=l2(reg))(x)
x = BatchNormalization()(x)
x = ReLU()(x)
# 16*16*128
x = Conv2D(filters, kernel_size=ksize, padding='same', strides=(1,1), use_bias=False, kernel_regularizer=l2(reg))(x)
x = BatchNormalization()(x)
x = Add()([x, skip])
x = ReLU()(x)

# 3rd block
# 8*8*256
filters=256
skip = Conv2D(filters, kernel_size=1, padding='same', strides=(2,2), use_bias=False, kernel_regularizer=l2(reg))(x)   # 16*16 -> 8*8 으로 크기를 조정해야 함
x = Conv2D(filters, kernel_size=ksize, padding='same', strides=(2,2), use_bias=False, kernel_regularizer=l2(reg))(x)
x = BatchNormalization()(x)
x = ReLU()(x)
# 8*8*256
x = Conv2D(filters, kernel_size=ksize, padding='same', strides=(1,1), use_bias=False, kernel_regularizer=l2(reg))(x)
x = BatchNormalization()(x)
x = Add()([x, skip])
x = ReLU()(x)

# 8*8*256
skip = x
x = Conv2D(filters, kernel_size=ksize, padding='same', strides=(1,1), use_bias=False, kernel_regularizer=l2(reg))(x)
x = BatchNormalization()(x)
x = ReLU()(x)
# 8*8*256
x = Conv2D(filters, kernel_size=ksize, padding='same', strides=(1,1), use_bias=False, kernel_regularizer=l2(reg))(x)
x = BatchNormalization()(x)
x = Add()([x, skip])
x = ReLU()(x)

# 4th block
# 4*4*512
filters=512
skip = Conv2D(filters, kernel_size=1, padding='same', strides=(2,2), use_bias=False, kernel_regularizer=l2(reg))(x)   # 8*8 -> 4*4 으로 크기를 조정해야 함
x = Conv2D(filters, kernel_size=ksize, padding='same', strides=(2,2), use_bias=False, kernel_regularizer=l2(reg))(x)
x = BatchNormalization()(x)
x = ReLU()(x)
# 4*4*512
x = Conv2D(filters, kernel_size=ksize, padding='same', strides=(1,1), use_bias=False, kernel_regularizer=l2(reg))(x)
x = BatchNormalization()(x)
x = Add()([x, skip])
x = ReLU()(x)

# 4*4*512
skip = x
x = Conv2D(filters, kernel_size=ksize, padding='same', strides=(1,1), use_bias=False, kernel_regularizer=l2(reg))(x)
x = BatchNormalization()(x)
x = ReLU()(x)
# 4*4*512
x = Conv2D(filters, kernel_size=ksize, padding='same', strides=(1,1), use_bias=False, kernel_regularizer=l2(reg))(x)
x = BatchNormalization()(x)
x = Add()([x, skip])
x = ReLU()(x)

# Pooling, Flatten
x = AveragePooling2D(pool_size=(4, 4))(x)         # 4*4*512 -> 1*1*512 로 변경
x = Flatten()(x)                                  # 1*1*512 -> 512 로 변경

# Fully-connected layer
output = Dense(10, activation=tf.nn.softmax)(x)
resnet = tf.keras.Model(input_, output)

data_augmentation = Sequential()
data_augmentation.add(ZeroPadding2D(4, input_shape=(32,32,3)))
data_augmentation.add(RandomCrop(32, 32))
data_augmentation.add(RandomFlip('horizontal'))
data_augmentation.add(RandomRotation(0.05))

model = Sequential()
model.add(data_augmentation)
model.add(resnet)

model.summary()
model.layers[0].summary()
model.layers[1].summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_20 (Sequential)   (None, 32, 32, 3)         0         
_________________________________________________________________
functional_21 (Functional)   (None, 10)                11179978  
Total params: 11,179,978
Trainable params: 11,172,170
Non-trainable params: 7,808
_________________________________________________________________
Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_10 (ZeroPaddi (None, 40, 40, 3)         0         
_________________________________________________________________
random_crop_10 (RandomCrop)  (None, 32, 32, 3)         0         
_________________________________________________________________
random_flip_10 (RandomFlip)  (None, 32, 32, 3)         0         
______________

In [50]:
# fit

if optimizer == 'sgd':
    optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=momentum)
elif optimizer == 'adam':
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
LRS_callback = LearningRateScheduler(lr_scheduler)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, callbacks=[LRS_callback, PlottingCallback()])

 111/1563 [=>............................] - ETA: 29:21 - loss: 11.8585 - acc: 0.1895

KeyboardInterrupt: 

In [49]:
model.summary()
model.evaluate(X_test,y_test)

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_20 (Sequential)   (None, 32, 32, 3)         0         
_________________________________________________________________
functional_21 (Functional)   (None, 10)                11179978  
Total params: 11,179,978
Trainable params: 11,172,170
Non-trainable params: 7,808
_________________________________________________________________
313/313 [==============================] - 48s 153ms/step - loss: 13.2033 - acc: 0.1163


[13.203301429748535, 0.11630000174045563]